In [1]:
from tqdm import tqdm

from db.postgres import PostgreSQLConnector
from db.graph import Neo4JConnector
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

In [2]:
engine = PostgreSQLConnector.create_from_config("../config.ini").create_engine()
neo4j = Neo4JConnector.create_from_config("../config.ini")

In [21]:
df = pd.read_sql_query(
    "SELECT * FROM challenger_matches WHERE not is_train LIMIT 1000", engine)
df

cmid  id           game_creation game_duration     game_id game_mode  \
0    107106   0 2020-01-13 06:11:54.990      00:23:23  4082200863   CLASSIC   
1    107110   0 2020-03-19 06:43:56.360      00:31:17  4228477227   CLASSIC   
2    107118   0 2020-02-10 06:54:41.245      00:15:16  4143018307   CLASSIC   
3    107124   0 2020-01-29 07:26:50.783      00:17:20  4117328217   CLASSIC   
4    107131   0 2020-01-23 07:32:48.660      00:15:38  4104221036   CLASSIC   
..      ...  ..                     ...           ...         ...       ...   
995  112079   0 2019-12-26 14:09:43.023      00:29:40  4048136387   CLASSIC   
996  112085   0 2020-02-27 15:58:07.843      00:25:51  4181305562   CLASSIC   
997  112091   0 2020-01-25 20:01:19.014      00:11:08  4110002415   CLASSIC   
998  112094   0 2020-03-18 01:26:30.646      00:19:35  4225345789      ARAM   
999  112097   0 2019-09-11 00:09:35.172      00:13:09  3855722755   CLASSIC   

        game_type   game_version  map_id  \
0    MATCHED_GAME  10.1.303.9385      11   
1    MATCHED_GAME  10.6.313.2784      11   
2    MATCHED_GAME  10.3.307.1028      11   
3    MATCHED_GAME  10.2.305.4739      11   
4    MATCHED_GAME  10.2.305.4739      11   
..            ...            ...     ...   
995  MATCHED_GAME  9.24.300.6382      11   
996  MATCHED_GAME  10.4.308.9400      11   
997  MATCHED_GAME  10.2.305.4739      11   
998  MATCHED_GAME  10.6.313.2784      12   
999  MATCHED_GAME  9.18.288.4125      11   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [25]:
y, y_hat = [],[]

query_tmpl = '''MATCH path = shortestPath(
            (p1:Player {accountID: "<p1accountID>", platformID: "<p1platformID>"})
            -[w:WINS_OVER *]->
            (p2:Player {accountID: "<p2accountID>", platformID: "<p2platformID>"}))
        RETURN [r in relationships(path) | r.rate] AS scores'''

for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    team0Wins = []
    for i in range(5):
        
        playerWins = []
        for j in range(5):
            try:
                player_1 = row["participant_identities"][i]["player"]
                player_2 = row["participant_identities"][j+5]["player"]
            except IndexError as e:
                print(row)
                continue
            
            q1 = query_tmpl \
                    .replace("<p1accountID>", player_1["accountId"]).replace("<p1platformID>", player_1["platformId"]) \
                    .replace("<p2accountID>", player_2["accountId"]).replace("<p2platformID>", player_2["platformId"])
            
            q2 = query_tmpl \
                .replace("<p1accountID>", player_2["accountId"]).replace("<p1platformID>", player_2["platformId"]) \
                .replace("<p2accountID>", player_1["accountId"]).replace("<p2platformID>", player_1["platformId"])
            
            r1 = neo4j.exec(q1)
            r2 = neo4j.exec(q2)
            
            if len(r1.records) == 0 and len(r2.records) == 0:
                pass
            elif len(r1.records) > 0 and len(r2.records) == 0:
                playerWins.append(np.average(r1.records[0]["scores"]))
            elif len(r1.records) == 0 and len(r2.records) > 0:
                playerWins.append((-1) * np.average(r2.records[0]["scores"]))
            else:
                pass
                # matchesWon.append(1 if len(r1.records[0]["r"]) > len(r2.records[0]["r"]) else 0)
        team0Wins.append(playerWins)
    
    if team0Wins != [[], [], [], [], []]:
        is_winning = 0
        for r in team0Wins:
            is_winning += sum(r)  
        
        if is_winning > 1 or is_winning < -1:
            y.append(row["participants"][0]["stats"]["win"])
            y_hat.append(is_winning > 0)
    
predictions = pd.DataFrame({"y": y, "y_hat": y_hat})
predictions

100%|██████████| 1000/1000 [03:37<00:00,  4.60it/s]


,y,y_hat
0,True,True
1,False,False
2,True,False
3,True,False
4,False,False
...,...,...
307,False,False
308,True,True
309,False,True
310,False,True


In [27]:
predictions_clean = predictions.dropna()
confusion_matrix(predictions_clean.y.values, predictions_clean.y_hat.values)

array([[82, 68],
       [75, 87]], dtype=int64)

In [28]:
from sklearn.metrics import classification_report

print(classification_report(predictions_clean.y.values, predictions_clean.y_hat.values))

              precision    recall  f1-score   support

       False       0.52      0.55      0.53       150
        True       0.56      0.54      0.55       162

    accuracy                           0.54       312
   macro avg       0.54      0.54      0.54       312
weighted avg       0.54      0.54      0.54       312


In [223]:

predictions_clean.y_hat.value_counts()

y_hat
False    13
True     11
Name: count, dtype: int64

In [ ]:
'''
        query = f"""MATCH 
            path = shortestPath(
                (p1:Player {{
                    accountID: "{row[f'team_0_player_{j}_account_id']}", 
                    platformID: "{row[f'team_0_player_{j}_platform_id']}"
                    }})
                -[w:WINS_OVER *]->
                (p2:Player {{
                    accountID: "{row[f'team_1_player_{j}_account_id']}", 
                    platformID: "{row[f'team_1_player_{j}_platform_id']}"}}))
                WHERE all(r IN relationships(path) WHERE r.prob_win >= 0)
            RETURN relationships(path) AS r"""
    
        result = neo4j.exec(query)
        print(result.records)
        
        if len(result.records) > 0:
            winning_prob = [r["prob_win"] for r in result.records[0]["r"]]
            winning_probs.append(np.average(winning_prob))
    
    y.append(1 if row["team_0_win"] else 0)
    
    if len(winning_probs) > 0:
        print(np.average(winning_probs))
        y_hat.append(1 if np.sum(winning_probs) > 0 else 0)
    else:
        y_hat.append(None)
    '''